### Dataset Loading

In [1]:
import os
import shutil
import pandas as pd

from tensorflow.keras.preprocessing.image import ImageDataGenerator

from PIL import Image

In [2]:
responses_path = r"correlation_assignment\responses.csv"

df = pd.read_csv(responses_path)    

df["id"] = df["id"].astype(str) + ".png"

df[:5]

,id,corr
0,arjskzyc.png,-0.457810
1,jrsnrkcp.png,-0.523113
2,hycqmekg.png,-0.179033
3,cerpyxxj.png,0.251509
4,cohtzuog.png,0.353992


In [3]:
img_directory = r"correlation_assignment\images"

In [4]:
image = Image.open(os.path.join(img_directory, df['id'][0]))

print(image.size)

(150, 150)


In [10]:
train_num = int(len(df) * 0.6) 
val_num = int(len(df) * 0.2) 

train_df = df[:train_num]
val_df = df[train_num: train_num + val_num]
test_df = df[train_num + val_num:]

print(len(train_df), len(val_df), len(test_df))

90000 30000 30000


In [13]:
data_generator = ImageDataGenerator(rescale=1./255)

train_generator = data_generator.flow_from_dataframe(
    dataframe=train_df,
    directory=img_directory,  
    x_col="id",
    y_col="corr",
    target_size=(150, 150),
    batch_size=32,
    class_mode="raw",
    color_mode="grayscale"
)

val_generator = data_generator.flow_from_dataframe(
    dataframe=val_df,
    directory=img_directory,
    x_col="id",
    y_col="corr",
    target_size=(150, 150),
    batch_size=32,
    class_mode="raw",
    color_mode="grayscale"
)

test_generator = data_generator.flow_from_dataframe(
    dataframe=test_df,
    directory=img_directory,
    x_col="id",
    y_col="corr",
    target_size=(150, 150),
    batch_size=32,
    class_mode="raw",
    color_mode="grayscale"
)

Found 90000 validated image filenames.
Found 30000 validated image filenames.
Found 30000 validated image filenames.


### Model Setup

In [14]:
import tensorflow as tf
from tensorflow.keras import layers, models

import matplotlib.pyplot as plt

In [18]:
model = models.Sequential()

model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 1)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='linear'))

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mse', 'mae'])

model.summary()


c:\Users\USER\Desktop\Code\Python\PicCollage\PicCollage_Assignment1\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)               │ (None, 148, 148, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 72, 72, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 36, 36, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 34, 34, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 73984)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 64)             │     4,735,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,790,849 (18.28 MB)

 Trainable params: 4,790,849 (18.28 MB)

 Non-trainable params: 0 (0.00 B)

### Model Training

In [ ]:
history = model.fit(
    train_generator,
    epochs=5,
    validation_data=val_generator
)

Epoch 1/5
1281/2813 ━━━━━━━━━━━━━━━━━━━━ 5:39 221ms/step - loss: 1.7045 - mae: 0.2047 - mse: 1.7045

In [26]:
loss = history.history['loss']
val_loss = history.history['val_loss']
mae = history.history['mae']
val_mae = history.history['val_mae']
epochs = range(1, 5+1)

In [30]:
plt.figure(figsize=(10, 4))

plt.subplot(1, 2, 1)
plt.plot(epochs, loss, 'bo-', label='Training Loss')
plt.plot(epochs, val_loss, 'r*-', label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training & Validation Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(epochs, mae, 'bo-', label='Training MAE')
plt.plot(epochs, val_mae, 'r*-', label='Validation MAE')
plt.xlabel('Epochs')
plt.ylabel('Mean Absolute Error (MAE)')
plt.title('Training & Validation MAE')
plt.legend()